<a href="https://colab.research.google.com/github/lhenriquesf/tcc_consumo_energia/blob/main/TCC_Analise_Consumo_Energia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
from google.colab import auth
auth.authenticate_user()
project_id = 'snappy-spanner-474217-i9'
print("Autenticação concluída.")


Autenticação concluída.


In [112]:
# 1. IMPORTS DO PROJETO
import pandas as pd
import pandas_gbq
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [113]:
# 2. CARREGAMENTO DOS DADOS (BIGQUERY)
query = """
SELECT
  TIMESTAMP_TRUNC(Time, HOUR) as DataHora,
  Appliance as Aparelho,
  AVG(CAST(Consumption_Watts AS FLOAT64)) as Consumo_Medio_Hora,
  EXTRACT(HOUR FROM Time) as Hora,
  EXTRACT(DAYOFWEEK FROM Time) as DiaSemana,
  EXTRACT(MONTH FROM Time) as Mes
FROM
  `tcc_consumo_energia.dados_limpos_house2`
GROUP BY 1, 2, 4, 5, 6
ORDER BY 1
"""

print("Executando query no BigQuery...")
df = pandas_gbq.read_gbq(query, project_id=project_id)

print(f"Dados carregados. Linhas: {df.shape[0]}")
df.head()

Executando query no BigQuery...
Downloading: 100%|██████████|
Dados carregados. Linhas: 102186


,DataHora,Aparelho,Consumo_Medio_Hora,Hora,DiaSemana,Mes
0,2013-09-17 22:00:00+00:00,Toaster,0.00,22,3,9
1,2013-09-17 22:00:00+00:00,Dishwasher,0.00,22,3,9
2,2013-09-17 22:00:00+00:00,Television,0.00,22,3,9
3,2013-09-17 22:00:00+00:00,Microwave,0.00,22,3,9
4,2013-09-17 22:00:00+00:00,Fridge_Freezer,33.08,22,3,9


In [98]:
# 3. PREPARAÇÃO E FEATURE ENGINEERING
df = df.sort_values(by=['DataHora'])

df['Consumo_Lag1'] = df.groupby('Aparelho')['Consumo_Medio_Hora'].shift(1)
df['Consumo_Lag24'] = df.groupby('Aparelho')['Consumo_Medio_Hora'].shift(24)

print(f"Linhas antes de limpar: {df.shape[0]}")
print(f"Linhas com buracos (NaN): {df['Consumo_Lag24'].isna().sum()}")

#Pega a média das últimas 3 horas para identificar tendência de subida/descida
df['Rolling_Mean_3h'] = df.groupby('Aparelho')['Consumo_Medio_Hora'].transform(lambda x: x.shift(1).rolling(window=3).mean())

#Remove linhas com NaN gerados pelos Lags
df = df.dropna()

# Melhora muito a capacidade do modelo de distinguir os padrões
df_model = pd.get_dummies(df, columns=['Aparelho'], prefix='App')

print("-" * 30)
print(f"Dados prontos! Linhas finais: {df_model.shape[0]}")
print(f"Total de Colunas (Features): {df_model.shape[1]}")

Linhas antes de limpar: 102186
Linhas com buracos (NaN): 216
------------------------------
Dados prontos! Linhas finais: 101970
Total de Colunas (Features): 17


In [99]:
# 4. DEFINIÇÃO DE TREINO E TESTE
features_base = ['Hora', 'DiaSemana', 'Mes', 'Consumo_Lag1', 'Consumo_Lag24', 'Rolling_Mean_3h']
features_app = [col for col in df_model.columns if 'App_' in col]
features = features_base + features_app
target = 'Consumo_Medio_Hora'

# Definida data de Corte (80% do tempo cronológico)
'''
dates = df_model['DataHora'].unique()
split_idx = int(len(dates) * 0.8)
split_date = dates[split_idx]
'''

split_date = "2015-02-01"

print(f"Data de corte definida em: {split_date}")
print("="*40)
# Faz o split usando máscaras de data (Mais preciso)
mask_train = df_model['DataHora'] < split_date
mask_test = df_model['DataHora'] >= split_date

X_train = df_model.loc[mask_train, features]
y_train = df_model.loc[mask_train, target]

X_test = df_model.loc[mask_test, features]
y_test = df_model.loc[mask_test, target]

print(f"Treino começa em: {df_model.loc[X_train.index, 'DataHora'].min()}")
print(f"Treino termina em: {df_model.loc[X_train.index, 'DataHora'].max()}")
print("="*40)
print(f"Teste começa em: {df_model.loc[X_test.index, 'DataHora'].min()}")

Data de corte definida em: 2015-02-01
Treino começa em: 2013-09-18 22:00:00+00:00
Treino termina em: 2015-01-31 23:00:00+00:00
Teste começa em: 2015-02-01 00:00:00+00:00


In [105]:
# 5. COMPARAÇÃO DE MODELOS (CÓDIGO DE SELEÇÃO)

# Definindo os modelos com configurações padrão
modelos_para_teste = {
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "k-Nearest Neighbors": KNeighborsRegressor(n_neighbors=5),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42)
}

lista_resultados = []

print("Iniciando avaliação comparativa completa...\n")

for nome_modelo, modelo in modelos_para_teste.items():
    # 1. Treinamento
    modelo.fit(X_train, y_train)

    # 2. Predição
    y_pred_teste = modelo.predict(X_test)

    # 3. Cálculo das Métricas (TODAS)
    r2 = r2_score(y_test, y_pred_teste)
    mae = mean_absolute_error(y_test, y_pred_teste)

    lista_resultados.append({
        "Algoritmo": nome_modelo,
        "R2": r2,
        "MAE (Erro Absoluto)": mae
    })
    print(f"{nome_modelo}: Métricas calculadas.")

# Criando a tabela final
df_comparativo = pd.DataFrame(lista_resultados)

# Ordenando pelo melhor R²
df_comparativo = df_comparativo.sort_values(by="R2", ascending=False)

print("\n" + "="*80)
print("TABELA DE JUSTIFICATIVA (SELEÇÃO DE MODELO)")
print("="*80)
display(df_comparativo)

Iniciando avaliação comparativa completa...

Decision Tree: Métricas calculadas.
k-Nearest Neighbors: Métricas calculadas.
Random Forest: Métricas calculadas.

TABELA DE JUSTIFICATIVA (SELEÇÃO DE MODELO)


,Algoritmo,R2,MAE (Erro Absoluto)
2,Random Forest,0.48,14.92
1,k-Nearest Neighbors,0.24,18.02
0,Decision Tree,-0.08,16.98


In [106]:
# 6. TREINAMENTO
print("Treinando Modelo Random Forest...")

rf = RandomForestRegressor(n_estimators=150, min_samples_leaf=5, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

pred_rf = rf.predict(X_test)

r2_rf = r2_score(y_test, pred_rf)
mae_rf = mean_absolute_error(y_test, pred_rf)

print("\n" + "="*40)
print(" RESULTADOS FINAIS - MODELO ESCOLHIDO")
print("="*40)
print(f"RANDOM FOREST:")
print(f"  R2 (Coeficiente de Determinação):  {r2_rf:.4f}")
print(f"  MAE (Erro absoluto):  {mae_rf:.2f} Watts")
print("="*40)

Treinando Modelo Random Forest...

 RESULTADOS FINAIS - MODELO ESCOLHIDO
RANDOM FOREST:
  R2 (Coeficiente de Determinação):  0.5970
  MAE (Erro absoluto):  13.85 Watts


In [88]:
# 7. TABELA FINAL COM AS PREVISOES
df_resultado = df.loc[X_test.index].copy()
df_resultado['Consumo_Real'] = y_test
df_resultado['Previsao_RandomForest'] = pred_rf

df_export = df_resultado[['DataHora', 'Aparelho', 'Consumo_Real', 'Previsao_RandomForest']]

tabela_destino = 'tcc_consumo_energia.tabela_previsoes_finais'

print(f"Salvando resultados em: {tabela_destino}...")

pandas_gbq.to_gbq(df_export,
                  tabela_destino,
                  project_id=project_id,
                  if_exists='replace')
print("\nTabela criada no BigQuery")

Salvando resultados em: tcc_consumo_energia.tabela_previsoes_finais...



100%|██████████| 1/1 [00:00<00:00, 8559.80it/s]


Tabela criada no BigQuery


In [114]:
import sys
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import sklearn

print("VERSÕES DAS FERRAMENTAS UTILIZADAS:")
print("="*35)
print(f"Python:     {sys.version.split()[0]}")
print(f"Pandas:     {pd.__version__}")
print(f"NumPy:      {np.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print(f"Seaborn:    {sns.__version__}")
print(f"Scikit-Learn (Sklearn): {sklearn.__version__}")
print("="*35)

VERSÕES DAS FERRAMENTAS UTILIZADAS:
Python:     3.12.12
Pandas:     2.2.2
NumPy:      2.0.2
Matplotlib: 3.10.0
Seaborn:    0.13.2
Scikit-Learn (Sklearn): 1.6.1


In [117]:
import google.colab
from google.cloud import bigquery

print("VERSÕES DO ECOSSISTEMA GOOGLE:")
print("="*40)
print(f"Google Colab (Lib Python):  {google.colab.__version__}")
print(f"BigQuery Client (API):      {bigquery.__version__}")
print(f"Google Cloud Core:          {google.cloud.core.__version__}")
print("="*40)

print("\nVERSÃO DO GCLOUD SDK (Ferramenta de Linha de Comando):")
!gcloud --version | head -n 1

VERSÕES DO ECOSSISTEMA GOOGLE:
Google Colab (Lib Python):  0.0.1a2
BigQuery Client (API):      3.38.0

VERSÃO DO GCLOUD SDK (Ferramenta de Linha de Comando):
Google Cloud SDK 549.0.0
